<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_04_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url_1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/corolla_1.csv'
url_2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/corolla_2.csv'
data_1 = pd.read_csv(url_1)
data_2 = pd.read_csv(url_2)

In [0]:
# [0]

# merge two tables
base = pd.merge(data_1, data_2, how='inner', on='ID')

# drop NAs
base = base.dropna()

In [22]:
# [1] ttest_ind

from scipy import stats
import math

# create input data set
auto = base[base['TransmissionType'] == 'Automatic']['Price']
manual = base[base['TransmissionType'] == 'Manual']['Price']

# perform ttest_ind
t_val, p_val = stats.ttest_ind(auto, manual, equal_var=True)

print(f'Answer [1] : {math.floor(p_val * 1000) / 1000}')

Answer [1] : 0.095


In [23]:
# [2] ANOVA

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import math

# analyze ANNOVA
ols_result = ols(formula='Price ~ C(Doors)', data=base).fit()
anova_table = anova_lm(ols_result)

# get result
f_val = anova_table['F'][0]
p_val = anova_table['PR(>F)'][0]

print(f'Answer [2] : {math.floor(f_val * 100) / 100}, ' +
      f'{math.floor(p_val * 10000) / 10000}')

Answer [2] : 6.65, 0.0001


In [29]:
# [3] corr, LinearRegression(OLS), list comprehension, enumerate, get_dummies, 
#     Series to DataFrame

import statsmodels.api as sm
import math

# find correlation
correl = base.corr(method='pearson')
correl_col = correl.columns[2:] # define X variables
correl_val = correl[correl_col].loc['Price'] # find corr values

# find variables with abs(corr) greater than 0.3
X_cols1 = [correl_col[i] for i, v in enumerate(correl_val) if abs(v) >= 0.3]

# create dummy variables
data3 = pd.get_dummies(base, columns=base.columns[7:], drop_first=True)
X_cols2 = data3.columns[7:]

# define X variables
X_cols = list(X_cols1) + list(X_cols2)

# create train set
train_X = data3[X_cols]
train_y = data3['Price']

# train
train_X = sm.add_constant(train_X)
model = sm.OLS(train_y, train_X)
reg_result = model.fit()

# find meaningful variables (P-value <= 0.05)
p_vals = reg_result.pvalues
reg_df = pd.DataFrame({'X': p_vals.index, 'p-value': p_vals.values})
effective_var = reg_df[reg_df['p-value'] <= 0.05]  # filter by p-value
effective_var = effective_var[effective_var['X'] != 'const']  # exclude const
count = effective_var['X'].count() # counting

# find Adjusted R^2
adj_r2 = reg_result.rsquared_adj

print(f'Answer [3] : {count}, {math.floor(adj_r2 * 1000) / 1000:.3f}')

Answer [3] : 5, 0.860


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [30]:
# [4]

import math

# create test data
# variables including p-val > 0.05, excluding corr < 0.3, including const
sample = [[1, 5, 50000, 143, 1170, 1, 0, 0, 0, 1, 0]]
sample_df = pd.DataFrame(sample, columns=train_X.columns)

# predict
pred = reg_result.predict(sample_df)

print(f'Answer [4] : {math.floor(pred)}')

Answer [4] : 21053
